<a href="https://colab.research.google.com/github/centuryglass/QtGLID3/blob/colab-refactor/colabFiles/Inpainting_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GLID-3-XL Inpainting UI backend server
Runs image generation operations for the inpainting UI, so that you can use the UI without a powerful GPU.

## Instructions:
1. Run all cells in this notebook, 
2. You will be asked to provide access to Google Drive. This is used to store your ngrok authentication token and optionally to load cached models. If you don't want to use drive, set `use_google_drive=False` below before running.
3. If you haven't run this before or you aren't using Google Drive, you will be prompted for an ngrok AuthToken in the first cell's output. This is required in order to connect to this server from the UI. You can sign up for an ngrok account for free at https://ngrok.com/, and you'll find your AuthToken at https://dashboard.ngrok.com/get-started/your-authtoken after logging in.
4. Wait for the server to start. This may take a few minutes, especially if you need to download model files.
5. Once you see `* Running on http://{Unique_session_id}.ngrok.io` in the output, the server is ready. Launch the inpainting UI, and enter in the `ngrok.io` link from the output when prompted for a server URL.


In [4]:
# Define settings, load models:

# Google Drive options:
# If use_google_drive=True, this will attempt to load your ngrok Authtoken and
# model files from Google Drive. If models aren't in drive they will be 
# downloaded normally.
use_google_drive=True
drive_model_dir="inpainting"

# If you're using google drive and haven't already uploaded models, set this
# to True and models will be uploaded to Drive so they can be loaded more
# quickly next time.  This will require something like 6-10 GB of available
# storage in Drive, depending on what models you're using.
save_missing_models_to_drive=False

# ML model options:
# model_path sets the primary model name. Any of the following options will be
# automatically downloaded from the appropriate URL:
#  'inpaint.pt':  The base inpainting model from glid-3-xl
#  'ongo.pt':     Fine-tuned on images from WikiArt by LAION-AI
#  'erlich.pt':   Fine-tuned on logos by LAION-AI
# If you want to use any other model finetuned from inpaint.pt, make sure it's
# already in Google Drive in the inpainting directory, or set model_url below to
# download it:
model_path = 'inpaint.pt'
model_url = None

# Set clip_guidance to true for greater accuracy but reduced speed.
# This requires Colab Pro, free Colab instances don't have enough GPU memory.
# Currently untested, as I don't have Colab Pro.
clip_guidance=False
clip_guidance_scale=150

# Set clip model to use, valid options are RN50, RN101, RN50x4, RN50x16, 
# RN50x64, ViT-B/32, ViT-B/16, ViT-L/14. Most of these are not working when
# clip_guidance is true, but ViT-L/14 and RN50x16 might work on Colab Pro.
clip_model_name = 'ViT-L/14'

# Changing these will subtly alter the image generation process in ways I
# haven't bothered to identify yet. You should just be able to leave them as-is
# and still get good results:
cutn=16
ddim=False
ddpm=False # Not working currently, leave this as False for now



# Setup ngrok, google drive(if enabled):
import os
drive_path='/content/gdrive/MyDrive/'
drive_model_dir="inpainting"
if use_google_drive:
    from google.colab import drive
    drive.mount('/content/gdrive')
    if drive_model_dir:
      drive_path = drive_path + drive_model_dir + '/'
      if not os.path.exists(drive_path):
        os.mkdir(drive_path)

# load authtoken from Drive if possible, otherwise prompt for input:
authtoken = None
if use_google_drive and os.path.isfile(drive_path + 'ngrok-Authtoken.txt'):
  with open(drive_path + 'ngrok-Authtoken.txt', 'r', newline='') as tokenFile:
    authtoken = tokenFile.read()
if authtoken is None:
  from getpass import getpass
  authtoken = getpass('Enter ngrok Authtoken:')
  if use_google_drive:
    with open(drive_path + 'ngrok-Authtoken.txt', 'w', newline='') as tokenFile:
      tokenFile.write(authtoken)

# install dependencies:
import os, sys
!pip install ipywidgets omegaconf>=2.0.0 pytorch-lightning>=1.0.8 torch-fidelity einops ftfy regex tqdm transformers flask flask_cors flask_ngrok pyngrok
!pip install git+https://github.com/openai/CLIP.git
%cd /content
for compVisDep in ['taming-transformers', 'latent-diffusion']:
  if not os.path.exists(compVisDep):
    !git clone https://github.com/CompVis/{compVisDep}.git
    !pip install -e {compVisDep}
    sys.path.append(f'/content/{compVisDep}')
if not os.path.exists('glid-3-xl-expanded-inpainting'):
  !git clone https://github.com/centuryglass/glid-3-xl-expanded-inpainting
%cd glid-3-xl-expanded-inpainting
!git fetch origin
!git checkout origin/colab-refactor
!pip install -e .

# download required models:
if use_google_drive:
    from google.colab import drive
    drive.mount('/content/gdrive')

for model in ['bert.pt', 'kl-f8.pt', model_path]:
    if use_google_drive and not os.path.isfile(model) and os.path.isfile(f'{drive_path}{model}'):
      !cp '{drive_path}{model}' .
    if not os.path.isfile(model):
      if model_url is not None:
        !wget -o '{model}' '{model_url}'
      elif model == "erlich.pt":
        !wget --continue -O erlich.pt https://huggingface.co/laion/erlich/resolve/main/model/ema_0.9999_120000.pt
      elif model == "ongo.pt":
        !wget https://huggingface.co/laion/ongo/resolve/main/ongo.pt
      else:  
        !wget 'https://dall-3.com/models/glid-3-xl/{model}'
    if os.path.isfile(model):
      print(f"model file '{model}' obtained.")
      if use_google_drive and save_missing_models_to_drive and not os.path.isfile(f'{drive_path}{model}'):
        !cp '{model}' '{drive_path}{model}'
        if os.path.isfile(f'{drive_path}{model}'):
          print(f'saved {model} to google drive')
        else:
          print(f'error: unable to save {model} to google drive')
    else:
      print(f"error: failed to get required model file '{model}'!")

# load all models: 
import torch
import gc
gc.collect()
device = torch.device('cuda:0')
from startup.load_models import loadModels
model_params, model, diffusion, ldm, bert, clip_model, clip_preprocess, normalize = loadModels(
    device,
    model_path=model_path,
    clip_model_name=clip_model_name,
    clip_guidance=clip_guidance,
    ddpm=ddpm,
    ddim=ddim)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-y_ckeec5
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-y_ckeec5
/content
/content/glid-3-xl-expanded-inpainting
HEAD is now at 630ff08 initial docs, sketch inputs, fixes
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/glid-3-xl-expanded-inpainting
  Attempting uninstall: guided-diffusion
    Found existing installation

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

loaded and configured BERT model from bert.pt


100%|███████████████████████████████████████| 890M/890M [00:13<00:00, 66.8MiB/s]


loaded and configured CLIP model from ViT-L/14


In [ ]:
from flask_ngrok import run_with_ngrok
from colabFiles.server import startServer
app = startServer(device, model_params, model, diffusion, ldm, bert, clip_model, clip_preprocess, normalize)
run_with_ngrok(app)
app.run()


Starting server...
 * Serving Flask app "colabFiles.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e2eb-34-143-208-14.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Jul/2022 16:20:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:22:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:22:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:23:08] "POST / HTTP/1.1" 200 -


  0%|          | 0/27 [00:00<?, ?it/s]

...

127.0.0.1 - - [16/Jul/2022 16:33:43] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:33:52] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:33:59] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:34:05] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:34:37] "POST / HTTP/1.1" 200 -


  0%|          | 0/27 [00:00<?, ?it/s]

127.0.0.1 - - [16/Jul/2022 16:34:40] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:34:48] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:34:56] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:35:04] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:35:32] "POST / HTTP/1.1" 200 -


  0%|          | 0/27 [00:00<?, ?it/s]

127.0.0.1 - - [16/Jul/2022 16:35:36] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:35:43] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:35:49] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2022 16:35:57] "GET //sample HTTP/1.1" 200 -
